In [1]:
import pandas as pd
import sys
sys.path.insert(1, "./rnn_text")
import train
import model
import helpers
import sys
sys.path.insert(1, "./feature-extraction/embed-extractor")
sys.path.insert(1, "./feature-extraction/vulgar-extractor")
from VulgarExtractor import VulgarExtractor
sys.path.insert(1, "./feature-extraction/twitter-parser")
from TwitterParser import TwitterParser
import classifiers
import pandas as pd
import numpy as np
import json
import torch
import torch.nn as nn
import helpers
from torch.autograd import Variable


In [2]:
df_tr = pd.read_pickle("output/simple/train_data_simple.pickle")
df_dev = pd.read_pickle("output/simple/dev_data_simple.pickle")
df_goldtest = pd.read_pickle("output/simple/goldtest_data_simple.pickle")

In [3]:
df_tr = df_tr['text']
df_dev = df_dev['text']
df_goldtest = df_goldtest['text']

In [4]:
import string
n_epochs = 2000
hidden_size = 20
learning_rate = 0.001


decoder = model.RNN(len(helpers.all_letters), hidden_size=hidden_size, output_size =3)
# Loss function can be changed as an argument?
decoder_optimizer = torch.optim.SGD(decoder.parameters(), learning_rate)


In [5]:
# need stuff from main.py to test transformation into Tensors

def normalize(column_name, df):
    std = df[column_name].std()
    norm_col = df[column_name].apply(lambda x: x - std)
    df[column_name] = norm_col

# builds the labels and vectorizations of given data
#if you want to fool around with including/excluding certain features and whatnot, this is the place to do it

def labels_and_vectors(file, index=0):
    df = pd.read_pickle(file)
    
    wordlist = VulgarExtractor.vulgarWords("feature-extraction/vulgar-extractor/badwords.txt") 
    dftext = df[['text']]
    result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
    df['isVulgar'] = result

    #word_embeddings = [ee.tweetVec(tagged_line) for tagged_line in df['text']]
    textlist = [txt.replace('\n','') for txt in df['text'].tolist()]
    tagged_sents = TwitterParser.tag(textlist)
    df['POS'] = tagged_sents

    processed_sents = []
    for tagged_sent in df['POS']:
        processed_words = []
        for word, tag in tagged_sent:
            if tag == 'U':
                processed_words.append('someurl')
            elif tag == '@':
                processed_words.append('@someuser')
            else:
                processed_words.append(word)
        sent = ' '.join(processed_words)
        processed_sents.append(sent)
    df['text'] = processed_sents

    word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in df['POS']]
    pos_count_list = [TwitterParser.pos_counts(tagged_line) for tagged_line in df['POS']]
    contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in df['POS']]
    contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in df['POS']]
    contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in df['POS']]
    contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in df['POS']]

    df['wordCount'] = word_counts
    df['posCounts'] = pos_count_list
    df['containsAdjective'] = contains_adjs
    df['containsURL'] = contains_urls
    df['containsEmoji'] = contains_emojis
    df['containsAbbreviation'] = contains_abbrevs
    #df['wordEmbedding'] = word_embeddings


    for i, tag in enumerate(TwitterParser.tagset):
        tag_counts = []
        for pos_counts in df['posCounts']:
            tag_counts.append(pos_counts[i])
        column_name = 'num_' + tag
        df[column_name] = tag_counts
        normalize(column_name, df)
        
    # Changes "true"/"false"/"unverified" to numeric values, just like the in the early cells

    df.loc[df.classification == 'true', 'classification'] = 1
    df.loc[df.classification == 'false', 'classification'] = 2
    df.loc[df.classification == 'unverified', 'classification'] = 0
    # getting the labels

    #removed containsURL
    attributes = ['text']
#     for tag in TwitterParser.tagset:
#         attributes.append('num_' + tag)

    labels = df['classification']
    labels = [l for l in labels]
    labels = np.array(labels)


    # getting the values as a list of lists
    values = df[attributes].values.tolist()
    #word_embedding_values = df['wordEmbedding'].values.tolist()


    #Below puts the tweet ID as a feature. Comment this out if you aren't using tweetID
    ###for i,index in enumerate(df.index):
    ###    dev_values[i].append(int(index))


# #UNCOMMENT THIS IN ORDER TO INCOPORATE WORD_EMBEDDINGS AGAIN
#     for i,d in enumerate(word_embedding_values):
#        values[i].extend(d)

    values = np.array(values)
    if index == 1:
        return df.index, values

    return labels, values

In [6]:
tr_labels, tr_values = labels_and_vectors('output/simple/train_data_simple.pickle')
indices, dev_values,= labels_and_vectors('output/simple/goldtest_data_simple.pickle', index=1)

In [7]:
tvs = [arr.tolist() for arr in tr_values]
dvs = [arr.tolist() for arr in dev_values]

tls = [l.item() for l in tr_labels]

In [8]:
tvs[0]

['Mike Brown was staying with his grandmother for the summer , who lived in the community . #Ferguson']

In [9]:
print(helpers.line_to_tensor(tvs[0][0]).size())


torch.Size([99, 1, 100])


In [10]:
def train_all():
    for i, label in enumerate(tls):
        tensor = helpers.line_to_tensor(tvs[i][0])  
        category_tensor = Variable(torch.LongTensor([label]))
        line_tensor = Variable(tensor)
        output, loss = train.train(category_tensor, line_tensor,decoder)
        decoder_optimizer.step()

In [11]:
for i in range(50):
    train_all()

In [13]:
predictions = []
for d in dvs:
    tensor = helpers.line_to_tensor(d)  
    line_tensor = Variable(tensor)
    output = decoder.predict(line_tensor)
    predictions.append(output)

In [14]:

predictions

    

#print(predictions)


[
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torc